In [1]:
from nufeb_tools import utils,plot
from scipy.integrate import odeint
import numpy as np
import pandas as pd
from itertools import combinations
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import time
from tqdm import tqdm
import h5py

In [2]:
x = utils.get_data(directory=r'D:\runs\Run_33_66_72_1_2021-06-24')

In [3]:
x.positions

,Timestep,ID,type,radius,x,y,z,biomass,time
0,0,95,2,4.450000e-07,0.000025,0.000018,0.000002,87.112534,0.000000
1,0,11,1,9.600000e-07,0.000097,0.000027,0.000004,1371.210192,0.000000
2,0,9,1,9.550000e-07,0.000193,0.000002,0.000005,1349.896428,0.000000
3,0,85,2,4.845000e-07,0.000213,0.000014,0.000005,112.429934,0.000000
4,0,75,2,4.795000e-07,0.000125,0.000060,0.000005,108.984929,0.000000
...,...,...,...,...,...,...,...,...,...
1509472,35000,23090,2,4.200998e-07,0.000369,0.000451,0.000010,73.292227,97.222222
1509473,35000,5557,2,4.937473e-07,0.000368,0.000452,0.000010,118.991185,97.222222
1509474,35000,31029,2,4.517007e-07,0.000368,0.000452,0.000010,91.107271,97.222222
1509475,35000,35847,2,4.201434e-07,0.000370,0.000452,0.000010,73.315084,97.222222


In [4]:
def collect(time,df=x.positions):
    fitness = pd.DataFrame(columns=['ID','Fitness'])
    for cell in tqdm(df[(df.Timestep==time)].ID):
        fitness = fitness.append(pd.DataFrame([[cell,x.get_fitness2(time,cell)]],columns=['ID','Fitness']),ignore_index=True)
    return fitness
#fit = collect(x,30000)

In [5]:
def collect(x,time):
    temp = x.positions
    IDs = temp[(temp.Timestep==time)].ID
    fits = [x.get_fitness2(time,cell) for cell in tqdm(IDs)]

    return IDs,fits
#IDs,fits = collect(x,30000)
#pd.DataFrame([IDs.values,fits]).T

In [6]:
from dask import delayed
import dask.multiprocessing
dask.config.set(scheduler='processes',num_workers=8)

In [11]:
#fits = delayed(x.collect_fitness)()
#fits.compute()

In [8]:
#x.collect_fitness()

In [9]:
time = 100
cell = 5

In [10]:
pd.DataFrame([[time,cell,x.get_fitness2(time,cell)]],columns=['Time','ID','Fitness'])

,Time,ID,Fitness
0,100,5,0.000017
